this script investigates the relationship between collaboration and amount of funding received by country-discpline pair

In [76]:
import pandas as pd
import seaborn as sns
from itertools import permutations
import numpy as np

In [4]:
pubs_path="../../data/NationalFunding/Data/DerivedData/CleanedRawData/pub.pkl"

In [6]:
pub_df = pd.read_pickle(pubs_path)

In [7]:
pub_df.head()

,id,year,dis,type,author,funder,author_distinct,funder_distinct,funded,IntCol,cofund,intfund
0,60694041,2017,9,1,"[Canada, United States]","[United States, United States]","[Canada, United States]",[United States],1,1,0,0
1,48573156,2012,89,1,[France],Not-Funded,[France],Not-Funded,0,0,0,0
2,55323671,2015,89,1,"[France, Chile]","[Chile, Chile, Chile, Chile, Chile]","[France, Chile]",[Chile],1,1,0,0
3,57633115,2016,31,1,"[Switzerland, United States]",[Switzerland],"[Switzerland, United States]",[Switzerland],1,1,0,0
4,64831413,2018,99,1,[China],"[China, China, China, China, China, China]",[China],[China],1,0,0,0


In [44]:
all_coll = []
for item in pub_df[['dis','author_distinct']].values:
    dis = item[0]
    cntrys = item[1]
    if len(cntrys)>1:
        cntry_pairs = list(permutations(cntrys,2))
        cntry_pairs_dis = [(*pair, dis) for pair in cntry_pairs]
        all_coll.extend(cntry_pairs_dis)

In [52]:
coll_df = pd.DataFrame(all_coll,columns=['cntrya','cntryb','dis'])
coll_df = coll_df.groupby(['cntrya','cntryb','dis']).size().reset_index()
coll_df = coll_df.rename(columns={0:'count'})

calculate the number of funding grants each country-discipline received from the source country

In [66]:
author_df = pub_df[pub_df.funder!='Not-Funded'][['author_distinct','funder','dis']].explode('author_distinct')
fund_df = author_df.explode('funder').groupby(
    ['author_distinct','funder','dis']).size().reset_index()
fund_df = fund_df.rename(columns={'author_distinct':'author_cntry',0:'fundcnt'})

In [67]:
coll_df.head(2)

,cntrya,cntryb,dis,count
0,Afghanistan,Algeria,41,1
1,Afghanistan,Algeria,44,2


In [69]:
fund_df=fund_df[fund_df.author_cntry!=fund_df.funder]

In [73]:
fund_df.head(2)

,author_cntry,funder,dis,fundcnt
1,Afghanistan,Australia,1,2
2,Afghanistan,Australia,44,13


In [80]:
meta = coll_df.merge(fund_df,left_on=['cntrya','cntryb','dis'],right_on=['author_cntry','funder','dis'],how='outer')

In [85]:
(meta['count'].fillna(0)+1).values

array([2., 3., 4., ..., 1., 1., 1.])

In [86]:
x=np.log((meta['count'].fillna(0)+1).values)
y=np.log((meta['fundcnt'].fillna(0)+1).values)
np.corrcoef(x,y)

array([[1.        , 0.78508678],
       [0.78508678, 1.        ]])